# Virtual Networking Device In Linux

Integrating with the raw TCP packets typically requires root/superuser access. To get around this (on Linux) we can create a proxy device that non-super users are allowed to communicate with directly.

Why use a virtual hardware device (TAP device)?

1. Using virtual hardware provides more control to freely assign IP and MAC addresses. 
2. It also avoids changing your hardware settings, which could affect its ability to connect to the network.

## Creating A Virtual Device

To create a TAP device called tap-rust, execute the following command in your Linux console:

`sudo ip tuntap add mode tap name tap-rust user $USER`

- `sudo` executes command as root user.
- `ip tuntap` tells ip that we're managing TUN/TAP devices.
- `mode tap` uses the TUN tunneling mode.
- `name tap-rust` gives the device a unique name.
- `user $USER` grants access to your non-root user account.

To confirm that our tap-rust device was added, we can use the `ip tuntap list` subcommand.

## Allocate An IP Address To The Device

We need to allocate an IP address to our device and tell our system to forward packets to it. The following command establishes a network device called tap-rust and activates it:

`sudo ip link set tap-rust up`

This command assigns the IP address 192.168.42.100 to the device:

`sudo ip addr add 192.168.42.100/24 dev tap-rust`

The following command enables internet packets to reach the source IP address mask (-s 192.168.42.100/24) by appending a rule (-A POSTROUTING) that dynamically maps IP addresses to a device (-j MASQUERADE):

`sudo iptables -t nat -A POSTROUTING -s 192.168.42.0/24 -j MASQUERADE`

This command instructs the kernel to enable IPv4 packet forwarding:

`sudo sysctl net.ipv4.ip_forward=1`

## Removing The Virtual Device

`sudo ip tuntap del mode tap name tap-rust`